# Question 1
(20 pts) Consider GF(2^8) used in AES with the irreducible polynomial p(x) = x^8+x^4+x^3+x+1. 

a.	(10 pts) Perform the following multiplication in GF(2^8):

(x^7+ x^6+ x^4+x^3+x^2+1) x (x^7+x^5+ x^3+ x^2+x) = ?

b.	(10 pts) Show that the inverse of (x^7+ x^6+ x^4+x^3+x^2+1) in GF(2^8) is (x^7+ x^6+x^5+x^4+x^3).


In [0]:
# I took this cell from http://pythonfiddle.com/binary-finite-field-multiplication/
from functools import reduce

# constants used in the multGF2 function
mask1 = mask2 = polyred = None

def setGF2(degree, irPoly):
    """Define parameters of binary finite field GF(2^m)/g(x)
       - degree: extension degree of binary field
       - irPoly: coefficients of irreducible polynomial g(x)
    """
    def i2P(sInt):
        """Convert an integer into a polynomial"""
        return [(sInt >> i) & 1
                for i in reversed(range(sInt.bit_length()))]    
    
    global mask1, mask2, polyred
    mask1 = mask2 = 1 << degree
    mask2 -= 1
    polyred = reduce(lambda x, y: (x << 1) + y, i2P(irPoly)[1:])
        
def multGF2(p1, p2):
    """Multiply two polynomials in GF(2^m)/g(x)"""
    p = 0
    while p2:
        if p2 & 1:
            p ^= p1
        p1 <<= 1
        if p1 & mask1:
            p1 ^= polyred
        p2 >>= 1
    return p & mask2

In [85]:
# Set the field
setGF2(8, 0b100011011)
# x^7+ x^6+ x^4+x^3+x^2+1 TIMES x^7+x^5+ x^3+ x^2+x 
print(bin(multGF2(0b011011101, 0b010101110)))


0b1100001


a) x^6 + x^5 + 1

B) The product of the first and second equation should be 1 for it to be the inverse

In [86]:

# x^7 + x^6 + x^4 + x^3 + x^2 + 1 TIMES x^7 + x^6 + x^5 + x^4 + x^3
print(bin(multGF2(0b011011101, 0b011111000)))


0b1


# Question 2
(20 pts) Consider ten digests in the attached file “rainbow_table.py”, each of which is the hash of a six-character password. Your mission is to find those passwords using the rainbow table given in the attached file “rainbowtable.txt”. Complete and submit the Python code in the file “rainbow_table.py” such that it finds and prints out the ten passwords corresponding to the digests

In [0]:
!pip install pycryptodome

In [57]:
# Do not forget to install pycryptodome if not already installed
# pip install pycryptodome

import random
from Crypto.Hash import SHA256
from Crypto import Random
import json

def Reduction(x, Alphabet, length):
  pwd = ""
  t = x
  size = len(Alphabet)
  for j in range(0,length):
    pwd += Alphabet[t%size]
    t = t//size
  return pwd  

Alphabet = {0:'A', 1:'B', 2:'C', 3:'D', 4:'E', 5:'F', 6:'G', 7:'H', 8:'I', 9:'J', 10:'K', 11:'L', 12:'M', 13:'N', 14:'O', 15:'P', 16:'Q', 17:'R', 18:'S', 19:'T', 20:'U', 21:'V', 22:'W', 23:'X', 24:'Y', 25:'Z', 26:'.', 27:',', 28:'!', 29:'?'}
alpha_len = len(Alphabet)
pwd_len = 6
pwd_space = alpha_len**pwd_len 
t = 2**16
m = 2*(pwd_space//t)
# Example for computing one link in the chain; i.e., pwd(i+1) = R(H(pwd(i)))
print("This is how you compute one link in the hash chain")
pwd_i = "OO,XR." # a sample password
hash = SHA256.new(pwd_i.encode('utf-8')) # hash it
digest = int.from_bytes(hash.digest(), byteorder='big') # convert the hash into an integer
pwd_i1 = Reduction(digest%pwd_space, Alphabet, pwd_len) # Reduce it
print("pwd(i) --> pwd(i+1): ", pwd_i, "-->",pwd_i1)
 
# Read the rainbow table
f = open("rainbowtable.txt","r")
Rainbow_Table = json.loads(f.read())
f.close()

# Check one link in the rainbow table
# Testing again
pwd = Rainbow_Table[0][0] # first password in the hash chain
for i in range(0, t):
  hash = SHA256.new(pwd.encode('utf-8')) # hash it
  digest = int.from_bytes(hash.digest(), byteorder='big') # convert the hash into an integer
  pwd = Reduction(digest%pwd_space, Alphabet, pwd_len) # Reduce it
if pwd == Rainbow_Table[0][1]:
  print("The test passed:)")
else:
  print("The test failed:(")


# Digests
digest = [0]*10
digest[0] = 100245009005103267899033614526941913880246373596564823340949840790844809103119
digest[1] = 19975499931938919928595451537096181331883854568747045850221901234691104582098
digest[2] = 113755495174055610876492464753048312831115306302701553827210127388606508241384
digest[3] = 91716148188179664717616297774779369080831030621153106066937072640936294082436
digest[4] = 114327333148588727761456040560697699459972533926123848855642209266882904981056
digest[5] = 114092167432998812840496186716627935081797792706490494942204367130254495731666
digest[6] = 38281036052010144447899334632289647459864065649722502224373489543446886678643
digest[7] = 44545357949490023150618582332141371853866888964826995324110314901909474805088
digest[8] = 38048259072653533075550911757874348323176766191918852427444568385091382449858
digest[9] = 35430391149852444211048461076529046528250550719267058178400921942117732723330

print("\The mission is to find six-character passwords that correspond to these digests:\n")
passwords = []
for d in range(len(digest)):
  # print(d + 1)
  # p0 = R(H)
  p_0 = Reduction(digest[d] % pwd_space, Alphabet, pwd_len)
  # compare p0 and pj,t−1 for j = 1, . . . , m
  found = False
  for j in range(1, len(Rainbow_Table)):
    if p_0 == Rainbow_Table[j][1]:
      found = True
      break
  if found:
    # print('j is', j)
    # print(Rainbow_Table[j])
    p_prv = Rainbow_Table[j][0]
    for i in range(t):
      hash = SHA256.new(p_prv.encode('utf-8'))  # hash it
      dig = int.from_bytes(hash.digest(), byteorder='big')  # convert the hash into an integer
      p_i = Reduction(dig % pwd_space, Alphabet, pwd_len)  # Reduce it
      p_prv = p_i
      if i == t-2:
        passwords.append(p_i)
        break
  else:
    FOUND = False
    t_step = 2
    while not FOUND:
      hash = SHA256.new(p_0.encode('utf-8'))  # hash it
      dig = int.from_bytes(hash.digest(), byteorder='big')  # convert the hash into an integer
      p_1 = Reduction(dig % pwd_space, Alphabet, pwd_len)  # Reduce it
      for j in range(1, len(Rainbow_Table)):
        if p_1 == Rainbow_Table[j][1]:
          FOUND = True
          break
      p_0 = p_1
      t_step += 1
    # print('t_step:', t_step)
    # print('j is', j)
    # print(Rainbow_Table[j])
    p_prv = Rainbow_Table[j][0]
    for i in range(t):
      hash = SHA256.new(p_prv.encode('utf-8'))  # hash it
      dig = int.from_bytes(hash.digest(), byteorder='big')  # convert the hash into an integer
      p_i = Reduction(dig % pwd_space, Alphabet, pwd_len)  # Reduce it
      p_prv = p_i
      if i == t - t_step:
        passwords.append(p_i)
        break
print(passwords)


This is how you compute one link in the hash chain
pwd(i) --> pwd(i+1):  OO,XR. --> CNTH,Y
The test passed:)
\The mission is to find six-character passwords that correspond to these digests:

['RAF!MX', 'IZSROR', '.I,BA,', 'AC.B,A', 'ZHKVHG', 'CMDLNJ', '?G.D,N', 'KOGSLE', 'AM!?OU', 'TKCQZA']


['RAF!MX', 'IZSROR', '.I,BA,', 'AC.B,A', 'ZHKVHG', 'CMDLNJ', '?G.D,N', 'KOGSLE', 'AM!?OU', 'TKCQZA']


# Question 3
Alice encrypts the private factors of the modulus using her public key. In order to increase security, she multiplies them with a random integer k (a process called blinding). Namely, she performs the following operations

cp = (kp)e mod n and cq = (kq)e mod n,



where 
n = 7473729494541494368852084160833887966786966146604539788038425546370721005202575292878010603718779143680279072437859979437027212668979951812104144364476251149631445677106574998958718100538368157559892086233484623658307080278702230379266872415713089871004732096589645883883977415506508347976001334807533323619602524830567712971581132341673584628698079736760920789626175805249978676215337039548212810503820392130214528748452439765014841123109184880811527461970066588991000432403773750048368279867220678747020767042484253902752403310560243017584585301433673020033953058809415002025715734068685302158511640316340598416777
e = 65537

a.	(10 pts) Explain why this is not secure as anyone who obtains cp or cq can factor n. 

b.	(20 pts) Factor n assuming 
cp =  6237092736280340368143892975246879700280574956823483459089826300838549326698371192841125256658301422335506071280990036134969958430140091821413942110999828019223701914534660496145184049436793233117527141710469587852896572612908205944874924666105935993339272072516489985048899803563277390008625395078803685108980714466378814566336644612044895653938680103088252710048504048952616260337657469904426834625756755178823712343160146527660412642787198955050511984140986234095815703702908475146055296882216504049042640234735532803592978062118777287332266867117935272468491447326467928608953581142170727780193982988625594898212
cq = 6792216459178320663134922113672614244133286795167205940387994163295121940464502516578964764558190390582955957319087527927606354995278787002827810252216604393602551619997059706389437721444109986118687258571558770491123542808098899449070164438070259725300867432155155552476715445078689763334188428800272552596349007005222835623783668243124391007979542657577232082178629086100963707200714469436928816142276093722080937180495045103388174875710073753498922326189147660992582560179759354313355942469090808651587693854760749192023182331797094543656070987815654088544115591473666250129130603910045357579318640054232668524224

and decrypt the following ciphertext 
cm = 2105477800955456357154507258188617457189736593253101663074145074953981395245071021806753010279680874168054072882966675195121143457746629686736359549415401549852363809597210114934407644579644329802951418430994896968996554627286997788295104796260715656641752826531876202868222659539111193689690587665763970475954283298561326716629497355710903035324921787171114135903504002506114489283933796611287777504398400559494459519397397254887260526830796723748881723043560212833332272718498671835837847760126707870313591817583090393736883523163338481442463241882375466824267640258884419635167163135753490195544303858749215914789

and print out the plaintext message. For the solution of this question complete and submit the attached file “DeterministicRSA.py”.


a) So if we know cp or cq we can take the gcd of one of them and n to find p or q. This is because when we multiply both equations we get cp * cq = k^(2e) * (pq)^e (mod n) which is 0 since pq = n cp * cq is a multiple of n hence taking the gcd of one of them and n will give us a factor of n. Once we found p or q we can find q or p, hence finding the factors of n. Then we are able to decrypt the message.

In [1]:
def egcd(a, b):
    x,y, u,v = 0,1, 1,0
    while a != 0:
        q, r = b//a, b%a
        m, n = x-u*q, y-v*q
        b,a, x,y, u,v = a,r, u,v, m,n
    gcd = b
    return gcd, x, y

def modinv(a, m):
    gcd, x, y = egcd(a, m)
    if gcd != 1:
        return None  # modular inverse does not exist
    else:
        return x % m



'''
Output: y^d mod n
'''
def binaryExp(y, d, n):
    s = y
    d = list(bin(d)[3:])
    for i in range(len(d)):
        s = (s*s) % n
        if d[i] == '1':
            s = (s*y) % n
    return s


# print(binaryExp(7,560,561))

# This is an example to show how the conversion between integers and bytes is done
message = b'Be yourself; everyone else is already taken.'
m = int.from_bytes(message, byteorder='big')
print("Message: ", m.to_bytes((m.bit_length()+7)//8, byteorder='big'))

n = 7473729494541494368852084160833887966786966146604539788038425546370721005202575292878010603718779143680279072437859979437027212668979951812104144364476251149631445677106574998958718100538368157559892086233484623658307080278702230379266872415713089871004732096589645883883977415506508347976001334807533323619602524830567712971581132341673584628698079736760920789626175805249978676215337039548212810503820392130214528748452439765014841123109184880811527461970066588991000432403773750048368279867220678747020767042484253902752403310560243017584585301433673020033953058809415002025715734068685302158511640316340598416777
e = 65537
    
# Alice encrypts her RSA private keys p and q
cp = 6237092736280340368143892975246879700280574956823483459089826300838549326698371192841125256658301422335506071280990036134969958430140091821413942110999828019223701914534660496145184049436793233117527141710469587852896572612908205944874924666105935993339272072516489985048899803563277390008625395078803685108980714466378814566336644612044895653938680103088252710048504048952616260337657469904426834625756755178823712343160146527660412642787198955050511984140986234095815703702908475146055296882216504049042640234735532803592978062118777287332266867117935272468491447326467928608953581142170727780193982988625594898212
cq = 6792216459178320663134922113672614244133286795167205940387994163295121940464502516578964764558190390582955957319087527927606354995278787002827810252216604393602551619997059706389437721444109986118687258571558770491123542808098899449070164438070259725300867432155155552476715445078689763334188428800272552596349007005222835623783668243124391007979542657577232082178629086100963707200714469436928816142276093722080937180495045103388174875710073753498922326189147660992582560179759354313355942469090808651587693854760749192023182331797094543656070987815654088544115591473666250129130603910045357579318640054232668524224
cm = 2105477800955456357154507258188617457189736593253101663074145074953981395245071021806753010279680874168054072882966675195121143457746629686736359549415401549852363809597210114934407644579644329802951418430994896968996554627286997788295104796260715656641752826531876202868222659539111193689690587665763970475954283298561326716629497355710903035324921787171114135903504002506114489283933796611287777504398400559494459519397397254887260526830796723748881723043560212833332272718498671835837847760126707870313591817583090393736883523163338481442463241882375466824267640258884419635167163135753490195544303858749215914789

# Your mission is to show that you can factor n, find p and q, and decrypt cm
print("Your code goes here")
p, _, _ = egcd(cp, n)
q = n//p
phi_n = (p-1) * (q-1)
d = modinv(e, phi_n)
m = binaryExp(cm, d, n)
print("Message: ", m.to_bytes((m.bit_length()+7)//8, byteorder='big'))

Message:  b'Be yourself; everyone else is already taken.'
Your code goes here
Message:  b'Insanity is doing the same thing, over and over again, but expecting different results.'


b) Insanity is doing the same thing, over and over again, but expecting different results.

# Question 4
Consider the following security game. Suppose that an attacker wants to decrypt the ciphertext c encrypted using the RSA algorithm and obtain the plaintext m, where c = me mod N. She knows neither the private key d nor the factorization of the modulus N. However, she can query an oracle (e.g., a program) with a ciphertext c’ != c, and receives the corresponding plaintext m’ = c’d mod N.

a.	(10 pts) The attacker can decrypt c and recover m. Show how. 


b.	(20 pts) Consider the following RSA parameters and a challenge ciphertext



N:  4131050804701947615474143330647520043409422542555438899772006619584945340119494649404129371817979723151949121241703378752722699538230573280931110089904937167907235670950600408473367355490703599182689592510968586723123472663548236788459038268157326078778911167528038386964502552412608060108007769524518942771622175289823182473018026393098835614963683062594807036906626036129123789566927291732854619902597726403125881968574446179918998140970652556709962611125425594329231466588467030961620469125134654566923297538042292332544406175518916678210686925936341811732104917605118575775329334829543752265141266783630832854187

e:  65537

c:  286011333479246811807036978298499172055797623822007560823628330407176989331751815848494737394310492374805419675958067102325270756298968875428244016352736458046904203412669134207806910534413220489710408452767455368567189168921003750345234473915004072109521598843246444074965165007427386987929674642591506744871181356247565764232892236100024802851129256309465367597958913617035234759551640734971193268573642824284719009304086710729931481730871492097430958962565736552256356850187124726045482360175673464358349365112652477377441584665739736701619949825887312761535693506114283123294171040950993106172221548552046342629

The RSA oracle is implemented as a Windows exe file (RSA_Oracle.exe) provided in the assignment package. If you are unable to run the RSA oracle, you can send your query to Erkay Savaş (accessible via erkay.savas@sabanciuniv.edu)

I challenge you to find m corresponding to the ciphertext c. Note that m contains a meaningful message. Convert it to bytes to discover what it is.


a) The attacker can choose an integer x which is relatively prime to n. Then they can generate a ciphertext c_x by taking the modulus of x to the power e. (c_x = x^e (mod n)). Next, they can multiply c_x with the given ciphertext and give it to the oracle. The oracle will return us with a message m times x to the power of e*d mod N. (c_x * c = (m*x)^(e*d) (mod n)). We know that d is the modular inverse of e so e*d = 1 and since x is relatively prime with n we can multiply each side to get our message. (c_x * c)/x = m (mod n)

In [43]:
n = 4131050804701947615474143330647520043409422542555438899772006619584945340119494649404129371817979723151949121241703378752722699538230573280931110089904937167907235670950600408473367355490703599182689592510968586723123472663548236788459038268157326078778911167528038386964502552412608060108007769524518942771622175289823182473018026393098835614963683062594807036906626036129123789566927291732854619902597726403125881968574446179918998140970652556709962611125425594329231466588467030961620469125134654566923297538042292332544406175518916678210686925936341811732104917605118575775329334829543752265141266783630832854187
c = 286011333479246811807036978298499172055797623822007560823628330407176989331751815848494737394310492374805419675958067102325270756298968875428244016352736458046904203412669134207806910534413220489710408452767455368567189168921003750345234473915004072109521598843246444074965165007427386987929674642591506744871181356247565764232892236100024802851129256309465367597958913617035234759551640734971193268573642824284719009304086710729931481730871492097430958962565736552256356850187124726045482360175673464358349365112652477377441584665739736701619949825887312761535693506114283123294171040950993106172221548552046342629
e = 65537
# find the first number that s relatively prime to n
x = 2
g, _, _ = egcd(n,x)
while (g != 1):
  x += 1
  g, _, _ = egcd(n,x)

c_x = (x**e) % n
print(c_x * c) 
# c_x * c -----> oracle
# oracle ------> res
res = 420085601177848199526077306644180397697070129576434437692078704800395153001958038786298923145675494839168462046
m = (res*modinv(x, n)) %n
print("Message: ", m.to_bytes((m.bit_length()+7)//8, byteorder='big'))





6518403098824205282166764259364737036386457897487070463813626296490370179735629351010551625868363779599919776249138991068103594742951150261261896957011307411940938232510865044487367870483513370257689914400369399789513794307637248442737714927701850295092934139940007304870093497830019878993026901574944865397283913036788474920654263972324698657146601982303050198161418301180410546598047967845030888583482624906046818214323098680068679301113850836786844517680537648411797519985932877762426685818181660574984065444206106926830426637325130484658524147059328010310406243740600557799686992732299245311552544497112401127693217618790550856043588229276399543296188489430518139877926701770350451370344974748682052646484745074633493680614225543892124632341209353108149831263072762806986166244998938323619163078007223890088111689025523727795380938333303232992004620837345193167628458927441410528870990224280797888581009099903670882309505610732134091666086530612941758374444404905241721717564265103339596192256437

b) You discovered my verry secret message:) Bravo